In [197]:
import pandas as pd
import numpy as np
from itertools import combinations
from functools import reduce
from collections import Iterable

# use for vertify 
from mlxtend.frequent_patterns import apriori, association_rules

In [198]:
df = pd.read_csv(
    '/home/sokhorn/sokhorn/dataSet/data/sample_data_set.csv', sep=',')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df


,InvoiceNo,StockCode,lower,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,white hanging heart t-light holder,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,white metal lantern,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,cream cupid hearts coat hanger,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,knitted union flag hot water bottle,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,red woolly hottie white heart.,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
5,536365,22752,set 7 babushka nesting boxes,SET 7 BABUSHKA NESTING BOXES,2,2010-12-01 08:26:00,7.65,17850,United Kingdom
6,536365,21730,glass star frosted t-light holder,GLASS STAR FROSTED T-LIGHT HOLDER,6,2010-12-01 08:26:00,4.25,17850,United Kingdom
7,536366,22633,hand warmer union jack,HAND WARMER UNION JACK,6,2010-12-01 08:28:00,1.85,17850,United Kingdom
8,536366,22632,hand warmer red polka dot,HAND WARMER RED POLKA DOT,6,2010-12-01 08:28:00,1.85,17850,United Kingdom
9,536367,84879,assorted colour bird ornament,ASSORTED COLOUR BIRD ORNAMENT,32,2010-12-01 08:34:00,1.69,13047,United Kingdom


In [199]:
sample_dataset = df[[
    'InvoiceNo',
    'StockCode',
    'Quantity'
]]
sample_dataset


,InvoiceNo,StockCode,Quantity
0,536365,85123A,6
1,536365,71053,6
2,536365,84406B,8
3,536365,84029G,6
4,536365,84029E,6
5,536365,22752,2
6,536365,21730,6
7,536366,22633,6
8,536366,22632,6
9,536367,84879,32


In [200]:
item_sets = (
    sample_dataset.groupby(['InvoiceNo', 'StockCode', ])['Quantity']
    .sum().unstack().reset_index().fillna(0)
    .set_index("InvoiceNo")
)
item_sets


StockCode,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
InvoiceNo,,,,,,,,,,,,,
536365,6.0,0.0,0.0,0.0,0.0,0.0,2.0,6.0,6.0,6.0,8.0,0.0,6.0
536366,0.0,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536367,0.0,0.0,0.0,6.0,6.0,8.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0
536368,0.0,0.0,0.0,6.0,6.0,8.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0
536369,0.0,0.0,0.0,6.0,6.0,8.0,0.0,0.0,0.0,0.0,0.0,32.0,0.0
536370,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536371,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [201]:
def en_code(x):
    if x > 0:
        return 1
    else:
        return 0

In [202]:
item_sets = item_sets.applymap(en_code)
item_sets

StockCode,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
InvoiceNo,,,,,,,,,,,,,
536365,1,0,0,0,0,0,1,1,1,1,1,0,1
536366,0,1,1,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,1,1,1,0,0,0,0,0,1,0
536368,0,0,0,1,1,1,0,0,0,0,0,1,0
536369,0,0,0,1,1,1,0,0,0,0,0,1,0
536370,0,0,0,0,0,1,0,0,0,0,0,0,0
536371,0,0,0,0,0,1,0,0,0,0,0,0,0


In [203]:
from itertools import combinations

In [204]:
cc = list(combinations(item_sets.columns, 1))


In [205]:
item_sets[sorted(cc[0])].sum() / len(item_sets)


StockCode
21730    0.142857
dtype: float64

In [206]:
item_sets


StockCode,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
InvoiceNo,,,,,,,,,,,,,
536365,1,0,0,0,0,0,1,1,1,1,1,0,1
536366,0,1,1,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,1,1,1,0,0,0,0,0,1,0
536368,0,0,0,1,1,1,0,0,0,0,0,1,0
536369,0,0,0,1,1,1,0,0,0,0,0,1,0
536370,0,0,0,0,0,1,0,0,0,0,0,0,0
536371,0,0,0,0,0,1,0,0,0,0,0,0,0


In [207]:
k_itemst = item_sets[['22748', '22749']]
k_itemst.loc[(k_itemst['22748'] > 0) & (k_itemst['22749'] > 0)]


StockCode,22748,22749
InvoiceNo,,
536367,1,1
536368,1,1
536369,1,1


In [208]:
item_sets[['22748', '22749']]


StockCode,22748,22749
InvoiceNo,,
536365,0,0
536366,0,0
536367,1,1
536368,1,1
536369,1,1
536370,0,1
536371,0,1


In [209]:
item_sets[['22748', '22749']].values


array([[0, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 1],
       [0, 1]])

In [210]:
support_count = 0
for i in item_sets[['22748', '22749']].values:
    if(np.all(i != 0)):
        support_count += 1
support_count


3

In [211]:
item_sets


StockCode,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
InvoiceNo,,,,,,,,,,,,,
536365,1,0,0,0,0,0,1,1,1,1,1,0,1
536366,0,1,1,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,1,1,1,0,0,0,0,0,1,0
536368,0,0,0,1,1,1,0,0,0,0,0,1,0
536369,0,0,0,1,1,1,0,0,0,0,0,1,0
536370,0,0,0,0,0,1,0,0,0,0,0,0,0
536371,0,0,0,0,0,1,0,0,0,0,0,0,0


In [212]:
item_sets[sorted(cc[1])].values[1] = 0

In [213]:
def calculate_support(k_itemst, trnaction_size=len(item_sets.index)):
    support_count = 0
    for i in k_itemst.values:
        if(np.all(i != 0)):
            support_count += 1
    return support_count / trnaction_size

**_Frequent Itemset Generation_**


In [214]:
min_support = 0.2
k = 1
item_sest_1 = combinations(item_sets.columns, k)
c1 = [
    {
        "Item": x,
        "Support":  calculate_support(item_sets[sorted(x)])
    }
    for x in sorted(item_sest_1)
    if calculate_support(item_sets[sorted(x)]) >= min_support
]
frequent_kitemset = c1
while(True):

    k += 1
    item_sest_k = combinations(item_sets.columns, k)

    c_k = [
        {
            "Item": x,
            "Support":  calculate_support(item_sets[sorted(x)])
        }
        for x in sorted(item_sest_k)
        if calculate_support(item_sets[sorted(x)]) >= min_support
    ] # generate untill we found ck equal to null 

    if(len(c_k) == 0):
        break
    else:
        frequent_kitemset.append(c_k)


flat_list = []
for sublist in frequent_kitemset:
    if(len(sublist) == 2):
        flat_list.append(sublist)
    else:
        for item in sublist:
            flat_list.append(item)
fk = pd.DataFrame(flat_list)

In [215]:
fk

,Item,Support
0,"(22745,)",0.428571
1,"(22748,)",0.428571
2,"(22749,)",0.714286
3,"(84879,)",0.428571
4,"(22745, 22748)",0.428571
5,"(22745, 22749)",0.428571
6,"(22745, 84879)",0.428571
7,"(22748, 22749)",0.428571
8,"(22748, 84879)",0.428571
9,"(22749, 84879)",0.428571


**Rule Generate tor**

In [216]:
fk  

,Item,Support
0,"(22745,)",0.428571
1,"(22748,)",0.428571
2,"(22749,)",0.714286
3,"(84879,)",0.428571
4,"(22745, 22748)",0.428571
5,"(22745, 22749)",0.428571
6,"(22745, 84879)",0.428571
7,"(22748, 22749)",0.428571
8,"(22748, 84879)",0.428571
9,"(22749, 84879)",0.428571


### old code  

**Other Function of Apriori**


In [217]:
def APRIORI_MY(data, min_support=0.04,  max_length=4):
    # Collecting Required Library
    import numpy as np
    import pandas as pd
    from itertools import combinations
    # Step 1:
    # Creating a dictionary to stored support of an itemset.
    support = {}
    L = list(data.columns)

    # Step 2:
    # generating combination of items with len i in ith iteration
    for i in range(1, max_length+1):
        c = set(combinations(L, i))

    # Reset "L" for next ith iteration
        L = set()
    # Step 3:
        # iterate through each item in "c"
        for j in list(c):

            sup = data.loc[:, j].product(axis=1).sum()/len(data.index)

            if sup > min_support:
                #print(sup, j)
                support[j] = sup

                # Appending frequent itemset in list "L", already reset list "L"
                L = list(set(L) | set(j))

    # Step 4: data frame with cols "items", 'support'
    result = pd.DataFrame(list(support.items()), columns=["Items", "Support"])
    return(result)



min_support = 0.2
item_sest_1 = combinations(item_sets.columns, 1)
c1 = [
    {
        "Item": x,
        "Support":  calculate_support(item_sets[sorted(x)])
    }
    for x in sorted(item_sest_1)
    if calculate_support(item_sets[sorted(x)]) >= min_support
]

k = 1
frequent_kitemset = c1
while(len(frequent_kitemset) != 0):

    k += 1
    item_sest_k = combinations(item_sets.columns, k)

    c_k = [
        {
            "Item": x,
            "Support":  calculate_support(item_sets[sorted(x)])
        }
        for x in sorted(item_sest_k)
        if calculate_support(item_sets[sorted(x)]) >= min_support
    ]

    if(len(c_k) == 0):
        break
    else:
        frequent_kitemset.append(c_k)


flat_list = []
for sublist in frequent_kitemset:
    if(len(sublist) == 2):
        flat_list.append(sublist)
    else:
        for item in sublist:
            flat_list.append(item)
fk = pd.DataFrame(flat_list)


In [218]:
# fk = APRIORI_MY(item_sets)
# fk

**Apriori**

In [219]:
item_sets

StockCode,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
InvoiceNo,,,,,,,,,,,,,
536365,1,0,0,0,0,0,1,1,1,1,1,0,1
536366,0,1,1,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,1,1,1,0,0,0,0,0,1,0
536368,0,0,0,1,1,1,0,0,0,0,0,1,0
536369,0,0,0,1,1,1,0,0,0,0,0,1,0
536370,0,0,0,0,0,1,0,0,0,0,0,0,0
536371,0,0,0,0,0,1,0,0,0,0,0,0,0


In [220]:
item_name =  item_sets.columns
item_name

Index(['21730', '22632', '22633', '22745', '22748', '22749', '22752', '71053',
       '84029E', '84029G', '84406B', '84879', '85123A'],
      dtype='object', name='StockCode')

In [221]:
items =  item_sets.to_numpy()
items

array([[1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]])

*Generating Itemset*

In [222]:
# generint first itemset 
C = {}
L = {}
Discard = {}
itemset_size = 1
Discard.update({itemset_size : []})
C.update({itemset_size : [ [f] for f in item_name ]})

In [223]:
item_sets

StockCode,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A
InvoiceNo,,,,,,,,,,,,,
536365,1,0,0,0,0,0,1,1,1,1,1,0,1
536366,0,1,1,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,1,1,1,0,0,0,0,0,1,0
536368,0,0,0,1,1,1,0,0,0,0,0,1,0
536369,0,0,0,1,1,1,0,0,0,0,0,1,0
536370,0,0,0,0,0,1,0,0,0,0,0,0,0
536371,0,0,0,0,0,1,0,0,0,0,0,0,0


**Generating RC Column**

In [224]:
item_sets['RC'] = 0
item_sets

StockCode,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A,RC
InvoiceNo,,,,,,,,,,,,,,
536365,1,0,0,0,0,0,1,1,1,1,1,0,1,0
536366,0,1,1,0,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,1,1,1,0,0,0,0,0,1,0,0
536368,0,0,0,1,1,1,0,0,0,0,0,1,0,0
536369,0,0,0,1,1,1,0,0,0,0,0,1,0,0
536370,0,0,0,0,0,1,0,0,0,0,0,0,0,0
536371,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [225]:
item_sets[item_sets.duplicated()]

StockCode,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A,RC
InvoiceNo,,,,,,,,,,,,,,
536368,0,0,0,1,1,1,0,0,0,0,0,1,0,0
536369,0,0,0,1,1,1,0,0,0,0,0,1,0,0
536371,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [226]:
item_np =  item_sets.values
item_np

array([[1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [227]:
# dup = []
# for i in range(len(item_np)):
#     item_i = item_np[i]
#     RC = 1
#     dup_list = []
#     for j in range(len(item_np)):
#         item_j = item_np[j]
#         if(i != j):
#             print(f'{i} : {item_i}, {j} : {item_j}')
#             if (item_i == item_j).all():
#                 RC += 1
#                 dup.append({
#                     item_i: RC
#                 })
#     for d in dup:
#         if (item_i != d):
#             dup.append({
#                 item_i: RC
#             })
#     print("\n")
# dup


In [228]:
# unique_rows, index = np.unique(item_np, axis=0)
# print(unique_rows)

we want to update RC Only with item itemset in row of tranction 
We increase only RC when item are duplicate 

In [229]:
item_np

array([[1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0],
       [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]])

In [230]:
item_rc = []
for i in range(len(item_np)):
    item_i = item_np[i]
    RC = 1
    aa = {
        "Item": item_i,
        "RC": RC
    }
    item_rc.append(aa)
    for j in range(len(item_np)):
        item_j = item_np[j]
        if i != j:
            if (item_i == item_j).all():
                RC += 1
                item_rc[i] = {
                    "Item": item_i,
                    "RC": RC
                }
item_rc


[{'Item': array([1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0]), 'RC': 1},
 {'Item': array([0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), 'RC': 1},
 {'Item': array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0]), 'RC': 3},
 {'Item': array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0]), 'RC': 3},
 {'Item': array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0]), 'RC': 3},
 {'Item': array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]), 'RC': 2},
 {'Item': array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]), 'RC': 2}]

In [231]:
df_rc =  pd.DataFrame(item_rc)
df_rc

,Item,RC
0,"[1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0]",1
1,"[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1
2,"[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0]",3
3,"[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0]",3
4,"[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0]",3
5,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",2
6,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]",2


In [232]:
df_rc['Item'] = df_rc['Item'].apply(lambda x : str(x))
df_rc.drop_duplicates(inplace=True)

In [233]:
def clean_str(str):
    str_list =  list((str.replace("[","").replace("]","").split(" ")))
    return [int (i) for i in str_list]

In [234]:
df_rc['Item'] =  df_rc['Item'].apply(clean_str)
split_df = pd.DataFrame(df_rc['Item'].tolist(), columns=item_sets.columns)

In [235]:
item_sets

StockCode,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A,RC
InvoiceNo,,,,,,,,,,,,,,
536365,1,0,0,0,0,0,1,1,1,1,1,0,1,0
536366,0,1,1,0,0,0,0,0,0,0,0,0,0,0
536367,0,0,0,1,1,1,0,0,0,0,0,1,0,0
536368,0,0,0,1,1,1,0,0,0,0,0,1,0,0
536369,0,0,0,1,1,1,0,0,0,0,0,1,0,0
536370,0,0,0,0,0,1,0,0,0,0,0,0,0,0
536371,0,0,0,0,0,1,0,0,0,0,0,0,0,0


In [236]:
split_df.set_index(df_rc.index, inplace=True)
split_df['RC'] = df_rc['RC']
split_df

StockCode,21730,22632,22633,22745,22748,22749,22752,71053,84029E,84029G,84406B,84879,85123A,RC
0,1,0,0,0,0,0,1,1,1,1,1,0,1,1
1,0,1,1,0,0,0,0,0,0,0,0,0,0,1
2,0,0,0,1,1,1,0,0,0,0,0,1,0,3
5,0,0,0,0,0,1,0,0,0,0,0,0,0,2


In [237]:
RC = split_df['RC'].values
RC

array([1, 1, 3, 2])

In [238]:
aa = split_df[['21730', '22752', '71053', '84029E', '84029G']].values
aa

array([[1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

In [239]:
def support_k_itemst(k_itemst):
    s = 0
    for i in range(len(k_itemst)):
        s += reduce(lambda a, b: a & b, k_itemst[i] & RC[i])
    return s

In [240]:
C = {}
L = {}
Discard = {}
itemset_size = 1

In [241]:
def count_ocurence(itemset, Tranctions):
    count = 0
    for i in range(len(Tranctions)):
        if set(itemset).issubset(set(Tranctions[i])):
            count += 1
    return count

In [242]:
def join_set_item(set_of_its, order_column_name):
    C = []
    for i in range(len(set_of_its)):
        for j in range(i + 1, len(set_of_its)):
            it_out = join_two_itemsets(
                set_of_its[i], set_of_its[j], order_column_name)
            if(len(it_out)) > 0:
                C.append(it_out)

    return C


In [243]:
def join_two_itemsets(it1, it2, order):
    it1.sort(key=lambda x : order.index(x))
    it2.sort(key=lambda x : order.index(x))

    for i in range(len(it1) - 1): # check befor the last one 
       if it1[i] != it2 [i] :
           return []
    
    if order.index(it1[-1]) < order.index(it2[-1]) :
        return [it1] + [it2[-1]]

    return []


In [349]:
def mergeKItemIntoOne(ab):
    result = []
    if(len(ab[0]) > 1):
        for item in ab:
            res = item[0] + item[1].split()
            result.append(res)
    else : return ab
    return result


In [383]:
def get_frequent(itemesets, min_support, prev_discard):
    L = []
    support_count = []
    new_discard = []

    k = len(prev_discard)
    for i in range(len(itemesets)):
        discard_before = False
        item = itemesets[i] 
        result = []
        for i in item:
            if isinstance(i, list):
                for j in i:
                    result.append(j)
            else:
                result.append(i)

        if k > 0:
            for it in prev_discard[k]:
                if set(it).issubset(set(result)):
                    discard_before = True
                    break

        if not discard_before:
            # count = support_k_itemst(split_df[mergeKItemIntoOne(item)].values)
            count = 1
            print(f"{split_df[mergeKItemIntoOne(item)]}\n\n")
            count = 1
            if count >= min_support:
                L.append(result)
                support_count.append(count)
            else:
                new_discard.append(result)
    return L, support_count, new_discard


f, supp, new_discard = get_frequent(C[2], min_support, Discard)

TypeError: unhashable type: 'list'

In [371]:
split_df[['84879', '85123A']]

StockCode,84879,85123A
0,0,1
1,0,0
2,1,0
5,0,0


In [248]:
def flatten(lis):
    for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
            for x in flatten(item):
                yield x
        else:        
            yield item

**Generating 1 itemse base on support count of row**

In [249]:
C = {}
L = {}
Discard = {}
itemset_size = 1
Discard.update({itemset_size : []})

In [250]:
combine = combinations(split_df.loc[:, split_df.columns != "RC"].columns, 1)
itemset_size = 1
min_support = 0.5
itemsets = []
for item in combine:
    coll_name = item[0].replace("[","").replace("]","").replace(",","")
    supp = support_k_itemst(split_df[[coll_name]].values)
    if supp >= min_support:
        itemsets.append([coll_name])
C.update({itemset_size : itemsets})

In [264]:
support_count = {}
f, supp, new_discard = get_frequent(
    C[itemset_size], min_support, Discard
)
Discard.update({itemset_size: new_discard})
L.update({itemset_size: f})
support_count.update({itemset_size: supp})

**To Be Continue**

In [265]:
order = list(split_df.loc[:, split_df.columns != 'RC'].columns)

In [ ]:
clean_k_itemset(C[2])

In [282]:
f, supp, new_discard = get_frequent(C[2], min_support, Discard)

TypeError: list indices must be integers or slices, not str

In [ ]:
k = itemset_size + 1
convergence = False
while not convergence:
    C.update({k: join_set_item(L[k - 1], order)})
    f, supp, new_discard = get_frequent(C[k], min_support, Discard)
    L.update({k: f})
    Discard.update({k: new_discard})
    support_count.update({k: supp})
    if(len(L[k]) == 0):
        convergence = True
    k += 1

===========================**Vertify with lib**=======================================

In [ ]:
itemset = apriori(split_df.loc[:,split_df.columns != "RC"], min_support=min_support, use_colnames=True)
itemset

**Apriori Algorithms**